# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_part = pd.read_csv("../output_data/weather_data.csv")
file_part.dropna()
file_part.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.1927,168.8643,283.97,89,97,1.70,NZ,1644699600
1,1,Chemin Grenier,-20.4872,57.4656,299.92,86,13,3.87,MU,1644699600
2,2,Arraial do Cabo,-22.9661,-42.0278,298.07,88,75,4.13,BR,1644699600
3,3,Rikitea,-23.1203,-134.9692,299.33,70,2,7.09,PF,1644699600
4,4,Albany,42.6001,-73.9662,278.27,58,57,5.00,US,1644699600


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
file_part.head()

humidity = file_part["Humidity"].astype(float) 
locations = file_part[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

weights= humidity

heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=max(file_part["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#294 - 300 K - 70-80 F
filtered_weather = file_part
filtered_weather = filtered_weather.loc[(filtered_weather["Max Temp"]>294) & (filtered_weather["Max Temp"]<300)]
filtered_weather = filtered_weather.loc[filtered_weather["Wind Speed"] < 10]
filtered_weather = filtered_weather.loc[filtered_weather["Cloudiness"] == 0]
filtered_weather = filtered_weather.dropna()
filtered_weather

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,East London,-33.0153,27.9116,297.67,89,0,4.42,ZA,1644699600
30,30,Ormara,25.2088,64.6357,294.27,63,0,5.67,PK,1644699600
32,32,Pacific Grove,36.6177,-121.9166,297.02,37,0,0.09,US,1644699600
64,64,Bubaque,11.2833,-15.8333,299.19,68,0,4.95,GW,1644699600
80,80,Chabahar,25.2919,60.6430,294.38,66,0,4.31,IR,1644699600
119,119,Hampton,37.0299,-76.3452,294.18,30,0,2.95,US,1644699600
125,125,Pisco,-13.7000,-76.2167,299.18,57,0,3.85,PE,1644699600
136,136,Wagar,16.1525,36.2032,294.18,57,0,6.80,SD,1644699600
146,146,Richards Bay,-28.7830,32.0377,296.39,86,0,4.32,ZA,1644699600
157,157,Rock Sound,24.9000,-76.2000,298.13,79,0,3.97,BS,1644699600


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_map = filtered_weather

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_map.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_map.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 27: East London.
Closest hotel is Tu Casa.
------------
Retrieving Results for Index 30: Ormara.
Closest hotel is Crystal Beach and Camping Resort Ormara.
------------
Retrieving Results for Index 32: Pacific Grove.
Closest hotel is Lovers Point Inn.
------------
Retrieving Results for Index 64: Bubaque.
Closest hotel is Casa Dora, Bubaque.
------------
Retrieving Results for Index 80: Chabahar.
Closest hotel is هتل گدروشیا.
------------
Retrieving Results for Index 119: Hampton.
Closest hotel is Embassy Suites by Hilton Hampton Convention Center.
------------
Retrieving Results for Index 125: Pisco.
Closest hotel is Hotel San Isidro Oficial.
------------
Retrieving Results for Index 136: Wagar.
Missing field/result... skipping.
------------
Retrieving Results for Index 146: Richards Bay.
Closest hotel is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Index 157: Rock Sound.
Closest hotel is Pink Corral Bay Property.
------------
Retr

In [8]:
hotel_map

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,27,East London,-33.0153,27.9116,297.67,89,0,4.42,ZA,1644699600,Tu Casa
30,30,Ormara,25.2088,64.6357,294.27,63,0,5.67,PK,1644699600,Crystal Beach and Camping Resort Ormara
32,32,Pacific Grove,36.6177,-121.9166,297.02,37,0,0.09,US,1644699600,Lovers Point Inn
64,64,Bubaque,11.2833,-15.8333,299.19,68,0,4.95,GW,1644699600,"Casa Dora, Bubaque"
80,80,Chabahar,25.2919,60.6430,294.38,66,0,4.31,IR,1644699600,هتل گدروشیا
119,119,Hampton,37.0299,-76.3452,294.18,30,0,2.95,US,1644699600,Embassy Suites by Hilton Hampton Convention Ce...
125,125,Pisco,-13.7000,-76.2167,299.18,57,0,3.85,PE,1644699600,Hotel San Isidro Oficial
136,136,Wagar,16.1525,36.2032,294.18,57,0,6.80,SD,1644699600,NaN
146,146,Richards Bay,-28.7830,32.0377,296.39,86,0,4.32,ZA,1644699600,BON Hotel Waterfront Richards Bay
157,157,Rock Sound,24.9000,-76.2000,298.13,79,0,3.97,BS,1644699600,Pink Corral Bay Property


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_map.iterrows()]
locations = hotel_map[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
locations = hotel_map[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

hotel_marker_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

fig.add_layer(markers)
fig.add_layer(hotel_marker_layer)
fig

#Answer


Figure(layout=FigureLayout(height='420px'))